In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression



KeyboardInterrupt: 

In [ ]:
lung_cancer_data = pd.read_csv('survey lung cancer.csv', encoding= 'latin-1')
lung_cancer_data

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER
0,M,69,1,2,2,1,1,2,1,2,2,2,2,2,2,YES
1,M,74,2,1,1,1,2,2,2,1,1,1,2,2,2,YES
2,F,59,1,1,1,2,1,2,1,2,1,2,2,1,2,NO
3,M,63,2,2,2,1,1,1,1,1,2,1,1,2,2,NO
4,F,63,1,2,1,1,1,1,1,2,1,2,2,1,1,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,F,56,1,1,1,2,2,2,1,1,2,2,2,2,1,YES
305,M,70,2,1,1,1,1,2,2,2,2,2,2,1,2,YES
306,M,58,2,1,1,1,1,1,2,2,2,2,1,1,2,YES
307,M,67,2,1,2,1,1,2,2,1,2,2,2,1,2,YES


XỬ LÝ DỮ LIỆU BƯỚC ĐẦU

CHECK COI DATA CÓ BỊ THIẾU DỮ LIỆU Ở COLUMN VÀ ROW NÀO KHÔNG

In [ ]:
# Check for missing values
missing_values = lung_cancer_data.isnull().sum()
missing_values

# Identify rows with any missing values
rows_with_missing_values = lung_cancer_data[lung_cancer_data.isnull().any(axis=1)]
rows_with_missing_values

# Display locations of missing values
missing_locations = lung_cancer_data[lung_cancer_data.isnull().any(axis=1)]
missing_locations

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER


=> KHÔNG CÓ COLUMN NÀO BỊ TRỐNG

CHUẨN HÓA DỮ LIỆU, BIẾN CHÚNG THÀNH CÙNG MỘT HỆ, MÀ CỤ THỂ Ở ĐÂY LÀ 0 VỚI 1

In [ ]:
# Encode categorical variables
lung_cancer_data['GENDER'] = lung_cancer_data['GENDER'].map({'M': 0, 'F': 1})
lung_cancer_data['LUNG_CANCER'] = lung_cancer_data['LUNG_CANCER'].map({'YES': 1, 'NO': 0})

# Assuming we scale all features except the target
features = lung_cancer_data.drop('LUNG_CANCER', axis=1)
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)
scaled_data = pd.DataFrame(scaled_features, columns=features.columns)
lung_cancer_data

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER
0,0,69,1,2,2,1,1,2,1,2,2,2,2,2,2,1
1,0,74,2,1,1,1,2,2,2,1,1,1,2,2,2,1
2,1,59,1,1,1,2,1,2,1,2,1,2,2,1,2,0
3,0,63,2,2,2,1,1,1,1,1,2,1,1,2,2,0
4,1,63,1,2,1,1,1,1,1,2,1,2,2,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,1,56,1,1,1,2,2,2,1,1,2,2,2,2,1,1
305,0,70,2,1,1,1,1,2,2,2,2,2,2,1,2,1
306,0,58,2,1,1,1,1,1,2,2,2,2,1,1,2,1
307,0,67,2,1,2,1,1,2,2,1,2,2,2,1,2,1


SPLIT DATA THÀNH HAI BÊN, 80% LÀ ĐỂ TRAIN, CÒN 20% LÀ ĐỂ TEST. HỆT NHƯ VIỆC CÓ 10 BÀI TOÁN, THẦY GIÁO CHỈ BAN 8 BÀI, VÀ BẠN PHẢI TỰ LÀM 2 BÀI CÒN LẠI ĐỂ LẤY ĐIỂM HS1 VẬY.

In [ ]:
from sklearn.model_selection import train_test_split

X = scaled_data
y = lung_cancer_data['LUNG_CANCER']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


OVERSAMPLING THE MINORITY-CLASS TO BALANCE THE TARGET VARIABLE

In [ ]:
from imblearn.over_sampling import SMOTE
# Example using SMOTE
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

In [ ]:
# Check balance of the target variable
y_train.value_counts(normalize=True)


LUNG_CANCER
1    0.5
0    0.5
Name: proportion, dtype: float64

TRAIN THE MODEL

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)




RandomForestClassifier(random_state=42)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# For AUC-ROC
y_pred_proba = model.predict_proba(X_test)[:, 1]
print("AUC-ROC:", roc_auc_score(y_test, y_pred_proba))


Accuracy: 0.967741935483871
Classification Report:
               precision    recall  f1-score   support

           0       0.50      0.50      0.50         2
           1       0.98      0.98      0.98        60

    accuracy                           0.97        62
   macro avg       0.74      0.74      0.74        62
weighted avg       0.97      0.97      0.97        62

AUC-ROC: 0.975


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid = GridSearchCV(RandomForestClassifier(), param_grid, refit=True, verbose=3)
grid.fit(X_train, y_train)
print(grid.best_params_)
best_model = grid.best_estimator_


Fitting 5 folds for each of 81 candidates, totalling 405 fits
[CV 1/5] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.893 total time=   0.0s
[CV 2/5] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.917 total time=   0.1s
[CV 3/5] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.976 total time=   0.0s
[CV 4/5] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.952 total time=   0.1s
[CV 5/5] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.976 total time=   0.0s
[CV 1/5] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=0.881 total time=   0.2s
[CV 2/5] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=0.917 total time=   0.1s
[CV 3/5] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=0.976 total time=  

In [ ]:
# Evaluate on the test set
y_test_pred = best_model.predict(X_test)
print("Final Test Accuracy:", accuracy_score(y_test, y_test_pred))
print("Final Test Classification Report:\n", classification_report(y_test, y_test_pred))

# Save the model
import joblib
joblib.dump(best_model, 'best_hyperparameter_tuned_model.pkl')



Final Test Accuracy: 0.967741935483871
Final Test Classification Report:
               precision    recall  f1-score   support

           0       0.50      0.50      0.50         2
           1       0.98      0.98      0.98        60

    accuracy                           0.97        62
   macro avg       0.74      0.74      0.74        62
weighted avg       0.97      0.97      0.97        62



['best_hyperparameter_tuned_model.pkl']

HÀM ĐỂ TÍNH GIÁ TRỊ CỦA TỪNG FEATURES (ƯU TIÊN VỚI KHÔNG ƯU TIÊN)